In [1]:
from sklearn.model_selection import KFold
from keras.metrics import categorical_accuracy
from sklearn.utils import shuffle
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
from keras import models
from keras import layers
from keras import optimizers
import numpy as np
import pickle as pk
import pandas as pd
import dask.dataframe
import csv
import pdb
import scipy.io

In [2]:
#numbers decribing the bci data
num_features = 64
number_of_people = 6
bci_channel_count = 11

In [4]:
#load bci feature and questoinnaire data from matlab files
def flatten_bci_feature_data(raw_data, num_features):
  data_size_x = int(raw_data.shape[0] / num_features)
  data_size_y = num_features * bci_channel_count

  data = np.array([])

  for person_index in range(0, number_of_people):
    temp_data = np.zeros((data_size_x, data_size_y))

    for i in range(temp_data.shape[0]):
        lr = i * num_features
        ur = (i + 1) * num_features
        temp_data[i] = raw_data[lr:ur, :, person_index].flatten('F')
      
    #pdb.set_trace()
    data = temp_data if person_index == 0 else np.append(data, temp_data, axis=0)
  
  return data

def stretch_arr(arr, length):
    repeat = np.repeat(arr, length // len(arr) + 1)
    return repeat[-length:]

#there are only ~27 values for the moving average (which will be "strected" to the number of rows BCI data)
def process_mov_avg_data(data_raw, num_rows_bci_data):
    #MinMax normalization
    data_raw = (data_raw - data_raw.min()) / (data_raw.max() - data_raw.min())
    
    return stretch_arr(data_raw, num_rows_bci_data)

num_features = 64

#"bci_feature_data" is x, questionnaire_data is y
bci_feature_data = np.array(scipy.io.loadmat("./drive/MyDrive/BCI_data/encoded_data.mat")["data"])
bci_feature_data = flatten_bci_feature_data(bci_feature_data, num_features)

questionnaire_data = scipy.io.loadmat("./drive/MyDrive/BCI_data/Questionnaire.mat")["Questionnaire"]["FearMovingAverage"][0][0]
questionnaire_data = np.array(questionnaire_data)
questionnaire_data = process_mov_avg_data(questionnaire_data, bci_feature_data.shape[0])

#"smooth" out survey data using savgol filter
window_size = 11
poly_order = 3
questionnaire_data = savgol_filter(questionnaire_data, window_size, poly_order)

In [ ]:
#method for building prediction model
def build_model(input_length):
  network = models.Sequential()

  network.add(layers.Dense(256, activation="relu", input_shape=(input_length,)))
  #network.add(layers.Dropout(0.5))
  network.add(layers.Dense(128, activation="relu"))
  #network.add(layers.Dropout(0.5))
  network.add(layers.Dense(64, activation="relu"))
  #network.add(layers.Dropout(0.5))
  network.add(layers.Dense(32, activation="relu"))
  network.add(layers.Dense(1))
      
  network.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])
  
  return network

In [ ]:
#train model with kfold validation
kfold_splits = 10
epochs = 10
batch_size = 4

x = bci_feature_data
y = questionnaire_data

graph_fold_results = True

kFold = KFold(n_splits=10, shuffle=True)

print('---------- CROSS FOLD VALIDATION ----------\n')

loss_arr = []
score_arr = []

fold_num = 1
for train, test in kFold.split(x, y):
  model = build_model(num_features * bci_channel_count)

  print(f'---------------------- FOLD NO {fold_num} TRAINING BEGIN ----------------------\n')

  model.fit(x[train], y[train], batch_size=batch_size, epochs=epochs)
  scores = model.evaluate(x[test], y[test])

  loss_arr.append(scores[0])
  loss_arr.append(scores[1])

  print(f'---------------------- FOLD NO {fold_num} TRAINING END ----------------------\n')

  if not graph_fold_results: 
      continue
  
  test_pred = model.predict(x[test])
  
  fold_results_fig = plt.figure()
  ax = fold_results_fig.add_axes([0.1,0.1,0.8,0.8])
  ax.set_title("Actual vs. Prediction")
  
  ax.plot(np.arange(y[test].shape[0]), y[test], label="actual")
  ax.plot(np.arange(test_pred.shape[0]), test_pred, label="prediction")
  ax.legend()

  fold_num += 1

print('------------ RESULTS PER FOLD ------------')
for i in range(len(loss_arr)):
    print(f'fold #{i}. Loss: {loss_arr[i]} - Score: {score_arr[i]}')
    print('--------------')
    
print('\n------- AVG ACROSS FOLDS -------')
print(f'Avg loss: {np.mean(loss_arr)} +/- {np.std(loss_arr)}')
print(f'Avg score: {np.mean(score_arr)} +/- {np.std(score_arr)}\n')